In [ ]:
!pip install git+https://github.com/sue-bin/soynlp.git

  Cloning https://github.com/sue-bin/soynlp.git to /tmp/pip-req-build-blh7a9fy
  Running command git clone -q https://github.com/sue-bin/soynlp.git /tmp/pip-req-build-blh7a9fy
  Created wheel for soynlp: filename=soynlp-0.0.493-cp36-none-any.whl size=419899 sha256=7466c0956c34a6f6533924334dfd19efb5771a6810a01a417c4bbf3687e5b12b
  Stored in directory: /tmp/pip-ephem-wheel-cache-x6qc4xcr/wheels/86/c5/94/4a36abe374aa83afed99292be54c9106ce1fdd7fd7bb5808df
Successfully built soynlp


In [ ]:
# import packages
import pandas as pd
import collections
from collections import Counter
from gensim.models import FastText
from tqdm import tqdm
from gensim.models import Word2Vec
from soynlp.word import WordExtractor
from soynlp.utils import DoublespaceLineCorpus
from soynlp.tokenizer import LTokenizer
from soynlp.utils import most_similar
from soynlp.normalizer import *
from soynlp.vectorizer import sent_to_word_contexts_matrix
from soynlp.word import pmi as pmi_func
import math
from soynlp.noun import LRNounExtractor_v2
import sys
pd.set_option('display.max_rows', None)
mod = sys.modules[__name__]

In [ ]:
for hln in highlight_num:
  try:
    #파일 읽기
    all = df_temp
    chat = getattr(mod,  'object_{}'.format(hln))

    display(all.head(10))

    #stopwords
    stopwords = []

    #채팅데이터 자모 제거 / 빈 채팅 제거
    all = all[all["comment"].apply(lambda x: isinstance(x, (str, bytes)))]
    all["comment"] = all["comment"].str.replace("[^^가-힣]","")
    all["comment"] = all["comment"].apply(lambda x: x.replace(" ",""))
    drop_list = []
    for i in range(len(all)):
      if all.iloc[i,0] == "":
        drop_list.append(i)

    all.drop(drop_list,inplace=True)
    all.reset_index(drop=True,inplace=True)
    chat_text_raw = chat[["comment"]]
    chat_text_df = chat_text_raw.copy()

    chat_text_df = chat_text_df[chat_text_df["comment"].apply(lambda x: isinstance(x, (str, bytes)))]
    chat_text_df["comment"] = chat_text_df["comment"].str.replace("[^가-힣]","") #한국어만 남김
    chat_text_df["comment"] = chat_text_df["comment"].apply(lambda x: x.replace(" ",""))
    #chat_text_df["comment"] = chat_text_df["comment"].apply(lambda x: emoticon_normalize(x, num_repeats=0))

    drop_list = []
    for i in range(len(chat_text_df)):
      if chat_text_df.iloc[i,0] == "":
        drop_list.append(i)

    chat_text_df.drop(drop_list,inplace=True)
    chat_text_df.reset_index(drop=True,inplace=True)


    #텍스트 고려 단어 추출 / 토크나이저 / 연관성 산출함수
    word_extractor = WordExtractor(
        min_frequency=10,
        min_cohesion_forward=0.3,
        min_right_branching_entropy=0.5
    )

    word_extractor.train(all["comment"])
    words = word_extractor.extract()

    import math

    def word_score(score):
        return (score.cohesion_forward * math.exp(score.right_branching_entropy))

    cohesion_score = {word:word_score(score) for word, score in words.items()}
    tokenizer = LTokenizer(scores=cohesion_score)

    x, idx2vocab = sent_to_word_contexts_matrix(all["comment"], windows=2,min_tf=5, tokenizer=tokenizer,dynamic_weight=True,verbose=False)

    pmi, px, py = pmi_func(
        x,
        min_pmi = 0.2,
        alpha = 0.0,
        beta = 0.75
    )

    #토큰화
    tokenized_data=[]
    for sentence in chat_text_df["comment"]:
      temp_X = tokenizer.tokenize(sentence)
      temp_X = [word for word in temp_X if not word in stopwords]
      tokenized_data.append(temp_X)

    #noun, token 데이터프레임 병합
    chat_text_df["tokenized"] = tokenized_data

    #토큰들 하나의 리스트로
    total_token = []
    for token in chat_text_df["tokenized"]:
      total_token = total_token+token

    #키워드 데이터프레임 만들기
    keyword_df = pd.DataFrame(columns=["keywords"])
    keyword_df["keywords"] = Counter(total_token).most_common(100)
    keyword_df["n"] = keyword_df["keywords"].apply(lambda x : x[1])
    keyword_df["keywords"] = keyword_df["keywords"].apply(lambda x : x[0])
    keyword_df.sort_values(by="n",ascending=False,inplace=True)

    #한글자는 조사일 가능성 높으므로 없애기
    drop_list = []
    for i in range(len(keyword_df)):
      if len(keyword_df["keywords"][i]) < 2:
        drop_list.append(i)
    keyword_df.drop(drop_list,inplace=True)  
    keyword_df.reset_index(drop=True,inplace=True)

    #썸네일용 키워드
    vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}

    for i in range(3):
      thumb_keyword = keyword_df.iloc[i,0]

      related = []
      for sim in most_similar(thumb_keyword, pmi, vocab2idx, idx2vocab):
        related.append(sim[0])
      print("<키워드 추출 결과>")
      print("빈도수 "+str(i+1)+"위 keyword: ",thumb_keyword,"  관련 단어: ",related[:3])

  except:
    print("!!error!!")

  print("\n\n\n")

-------------------------------주펄1-------------------------------


,timeline,userId,comment
0,1900-01-01 00:00:00,<11tlalsehrwo>,치마
1,1900-01-01 00:00:01,<eedaee>,야호
2,1900-01-01 00:00:01,<결핍소년>,치마
3,1900-01-01 00:00:02,<lililili1li>,침하
4,1900-01-01 00:00:03,<버디오>,침하
5,1900-01-01 00:00:03,<직플릭>,침하
6,1900-01-01 00:00:03,<여기지금>,침하
7,1900-01-01 00:00:03,<불전이굴>,얼마나
8,1900-01-01 00:00:04,<무친너구리섬>,치마!
9,1900-01-01 00:00:04,<eedaee>,앗


training was done. used memory 0.174 Gb
all cohesion probabilities was computed. # words = 1324
all branching entropies was computed # words = 696
all accessor variety was computed # words = 696
<키워드 추출 결과>
빈도수 1위 keyword:  침타래   관련 단어:  ['가지', '교권', '분재']
<키워드 추출 결과>
빈도수 2위 keyword:  침럼프   관련 단어:  ['가지', '교권', '분재']
<키워드 추출 결과>
빈도수 3위 keyword:  꿀타래   관련 단어:  ['가지', '교권', '분재']




-------------------------------주펄2-------------------------------


,timeline,userId,comment
0,1900-01-01 00:00:00,<11tlalsehrwo>,치마
1,1900-01-01 00:00:01,<eedaee>,야호
2,1900-01-01 00:00:01,<결핍소년>,치마
3,1900-01-01 00:00:02,<lililili1li>,침하
4,1900-01-01 00:00:03,<버디오>,침하
5,1900-01-01 00:00:03,<직플릭>,침하
6,1900-01-01 00:00:03,<여기지금>,침하
7,1900-01-01 00:00:03,<불전이굴>,얼마나
8,1900-01-01 00:00:04,<무친너구리섬>,치마!
9,1900-01-01 00:00:04,<eedaee>,앗


training was done. used memory 0.183 Gb
all cohesion probabilities was computed. # words = 1324
all branching entropies was computed # words = 696
all accessor variety was computed # words = 696
<키워드 추출 결과>
빈도수 1위 keyword:  아이스버거   관련 단어:  ['가지', '교권', '분재']
<키워드 추출 결과>
빈도수 2위 keyword:  이가   관련 단어:  ['가지', '교권', '분재']
<키워드 추출 결과>
빈도수 3위 keyword:  가지   관련 단어:  ['찌개', '킹', '킹은']




-------------------------------주펄3-------------------------------


...

training was done. used memory 0.192 Gb
all cohesion probabilities was computed. # words = 288
all branching entropies was computed # words = 204
all accessor variety was computed # words = 204
<키워드 추출 결과>
빈도수 1위 keyword:  고모   관련 단어:  ['무야호', '아', '아이고']
<키워드 추출 결과>
빈도수 2위 keyword:  나는   관련 단어:  ['아', '잇힝', '내가']
<키워드 추출 결과>
빈도수 3위 keyword:  시적허용   관련 단어:  ['아', '잇힝', '내가']




-------------------------------침착맨단독5-------------------------------


,timeline,userId,comment
0,1900-01-01 00:00:06,<앙앵잉>,오오오
1,1900-01-01 00:00:08,<yonumsiki>,??
2,1900-01-01 00:00:09,<앙앵잉>,왔다!!!!
3,1900-01-01 00:00:13,<NosPawn13>,침하
4,1900-01-01 00:00:15,<말년후배>,머냐구!!!
5,1900-01-01 00:00:15,<NosPawn13>,오셨구먼유
6,1900-01-01 00:00:16,<앙앵잉>,침하
7,1900-01-01 00:00:20,<굥원잉>,뭐야!!!!
8,1900-01-01 00:00:21,<Nightbot>,@침착맨
9,1900-01-01 00:00:21,<헤물맨>,치마


training was done. used memory 0.192 Gb
all cohesion probabilities was computed. # words = 288
all branching entropies was computed # words = 204
all accessor variety was computed # words = 204
<키워드 추출 결과>
빈도수 1위 keyword:  고모   관련 단어:  ['무야호', '아', '아이고']
<키워드 추출 결과>
빈도수 2위 keyword:  우서   관련 단어:  ['아', '잇힝', '내가']
<키워드 추출 결과>
빈도수 3위 keyword:  웃참   관련 단어:  ['아', '잇힝', '내가']




-------------------------------침착맨단독6-------------------------------


,timeline,userId,comment
0,1900-01-01 00:00:06,<앙앵잉>,오오오
1,1900-01-01 00:00:08,<yonumsiki>,??
2,1900-01-01 00:00:09,<앙앵잉>,왔다!!!!
3,1900-01-01 00:00:13,<NosPawn13>,침하
4,1900-01-01 00:00:15,<말년후배>,머냐구!!!
5,1900-01-01 00:00:15,<NosPawn13>,오셨구먼유
6,1900-01-01 00:00:16,<앙앵잉>,침하
7,1900-01-01 00:00:20,<굥원잉>,뭐야!!!!
8,1900-01-01 00:00:21,<Nightbot>,@침착맨
9,1900-01-01 00:00:21,<헤물맨>,치마


training was done. used memory 0.192 Gb
all cohesion probabilities was computed. # words = 288
all branching entropies was computed # words = 204
all accessor variety was computed # words = 204
<키워드 추출 결과>
빈도수 1위 keyword:  앵콜   관련 단어:  ['아', '잇힝', '내가']
<키워드 추출 결과>
빈도수 2위 keyword:  뇌절   관련 단어:  ['아', '잇힝', '내가']
<키워드 추출 결과>
빈도수 3위 keyword:  손한번   관련 단어:  ['아', '잇힝', '내가']




-------------------------------침착맨단독7-------------------------------
!!error!!




-------------------------------침착맨단독8-------------------------------
!!error!!






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
all = pd.read_csv("/content/drive/Shareddrives/20-2 ybigta project - 유튜브 썸네일 자동제작 프로그램/하이라이트 채팅로그_new/주펄/주펄chat(total).csv")

NameError: ignored